In [46]:
def parse(input):
  lines = [l.strip().split('~') for l in open(input, 'r').readlines()]
  return [(tuple(map(int, a.split(','))), tuple(map(int, b.split(',')))) for a, b in lines]

parse('sample')

[((1, 0, 1), (1, 2, 1)),
 ((0, 0, 2), (2, 0, 2)),
 ((0, 2, 3), (2, 2, 3)),
 ((0, 0, 4), (0, 2, 4)),
 ((2, 0, 5), (2, 2, 5)),
 ((0, 1, 6), (2, 1, 6)),
 ((1, 1, 8), (1, 1, 9))]

In [47]:
import numpy as np

def first_part(input):
  bricks = []
  maxes = (0, 0, 0)
  for i, ((min_x, min_y, min_z), (max_x, max_y, max_z)) in enumerate(input):
    maxes = (max(maxes[0], max_x + 1), max(maxes[1], max_y + 1), max(maxes[2], max_z + 1))
    bricks.append([[min_x, min_y, min_z], [max_x, max_y, max_z]])
  
  space = np.zeros(maxes, dtype=int)

  for ((min_x, min_y, min_z), (max_x, max_y, max_z)) in input:
    for x, y, z in np.ndindex((max_x - min_x + 1, max_y - min_y + 1, max_z - min_z + 1)):
      space[min_x + x, min_y + y, min_z + z] = 1

  bricks.sort(key=lambda b: b[0][2])

  # settle bricks
  for brick in bricks:
    slice = space[brick[0][0]:brick[1][0] + 1, brick[0][1]:brick[1][1] + 1, brick[0][2]:brick[1][2] + 1]
    assert np.all(slice == 1)

    space[brick[0][0]:brick[1][0] + 1, brick[0][1]:brick[1][1] + 1, brick[0][2]:brick[1][2] + 1] = 0
    while brick[0][2] - 1 > 0 and np.all(space[brick[0][0]:brick[1][0] + 1, brick[0][1]:brick[1][1] + 1, brick[0][2] - 1:brick[1][2]] == 0):
      brick[0][2] -= 1
      brick[1][2] -= 1
    space[brick[0][0]:brick[1][0] + 1, brick[0][1]:brick[1][1] + 1, brick[0][2]:brick[1][2] + 1] = 1

  result = len(bricks)
  for i in range(len(bricks)):
    space[bricks[i][0][0]:bricks[i][1][0] + 1, bricks[i][0][1]:bricks[i][1][1] + 1, bricks[i][0][2]:bricks[i][1][2] + 1] = 0
    for j in range(i + 1, len(bricks)):
      would_move = False

      space[bricks[j][0][0]:bricks[j][1][0] + 1, bricks[j][0][1]:bricks[j][1][1] + 1, bricks[j][0][2]:bricks[j][1][2] + 1] = 0
      if bricks[j][0][2] - 1 > 0 and np.all(space[bricks[j][0][0]:bricks[j][1][0] + 1, bricks[j][0][1]:bricks[j][1][1] + 1, bricks[j][0][2] - 1:bricks[j][1][2]] == 0):
        would_move = True
      space[bricks[j][0][0]:bricks[j][1][0] + 1, bricks[j][0][1]:bricks[j][1][1] + 1, bricks[j][0][2]:bricks[j][1][2] + 1] = 1

      if would_move:
        result -= 1
        break

    space[bricks[i][0][0]:bricks[i][1][0] + 1, bricks[i][0][1]:bricks[i][1][1] + 1, bricks[i][0][2]:bricks[i][1][2] + 1] = 1
  return result

assert first_part(parse('sample')) == 5
first_part(parse('input'))

443

In [57]:
import numpy as np
from copy import deepcopy

def second_part(input):
  bricks = []
  maxes = (0, 0, 0)
  for i, ((min_x, min_y, min_z), (max_x, max_y, max_z)) in enumerate(input):
    maxes = (max(maxes[0], max_x + 1), max(maxes[1], max_y + 1), max(maxes[2], max_z + 1))
    bricks.append([[min_x, min_y, min_z], [max_x, max_y, max_z]])
  
  space = np.zeros(maxes, dtype=int)

  for ((min_x, min_y, min_z), (max_x, max_y, max_z)) in input:
    for x, y, z in np.ndindex((max_x - min_x + 1, max_y - min_y + 1, max_z - min_z + 1)):
      space[min_x + x, min_y + y, min_z + z] = 1

  bricks.sort(key=lambda b: b[0][2])

  # settle bricks
  for brick in bricks:
    slice = space[brick[0][0]:brick[1][0] + 1, brick[0][1]:brick[1][1] + 1, brick[0][2]:brick[1][2] + 1]
    assert np.all(slice == 1)

    space[brick[0][0]:brick[1][0] + 1, brick[0][1]:brick[1][1] + 1, brick[0][2]:brick[1][2] + 1] = 0
    while brick[0][2] - 1 > 0 and np.all(space[brick[0][0]:brick[1][0] + 1, brick[0][1]:brick[1][1] + 1, brick[0][2] - 1:brick[1][2]] == 0):
      brick[0][2] -= 1
      brick[1][2] -= 1
    space[brick[0][0]:brick[1][0] + 1, brick[0][1]:brick[1][1] + 1, brick[0][2]:brick[1][2] + 1] = 1

  result = 0
  for i in range(len(bricks)):
    s = space.copy()
    b = deepcopy(bricks)
    falling = set()
    
    s[b[i][0][0]:b[i][1][0] + 1, b[i][0][1]:b[i][1][1] + 1, b[i][0][2]:b[i][1][2] + 1] = 0
    for j in range(i + 1, len(bricks)):
      s[b[j][0][0]:b[j][1][0] + 1, b[j][0][1]:b[j][1][1] + 1, b[j][0][2]:b[j][1][2] + 1] = 0
      while b[j][0][2] - 1 > 0 and np.all(s[b[j][0][0]:b[j][1][0] + 1, b[j][0][1]:b[j][1][1] + 1, b[j][0][2] - 1:b[j][1][2]] == 0):
        b[j][0][2] -= 1
        b[j][1][2] -= 1
        falling.add(j)
      s[b[j][0][0]:b[j][1][0] + 1, b[j][0][1]:b[j][1][1] + 1, b[j][0][2]:b[j][1][2] + 1] = 1

    s[b[i][0][0]:b[i][1][0] + 1, b[i][0][1]:b[i][1][1] + 1, b[i][0][2]:b[i][1][2] + 1] = 1
    result += len(falling)
  return result

assert second_part(parse('sample')) == 7
second_part(parse('input'))

69915